In [1]:
%load_ext autoreload
%autoreload 2
# set PYTORCH_ENABLE_MPS_FALLBACK=1 in your environment

import os 
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
import yaml
import torch
import lightning_toolbox as ltb
import lightning
import dypy as dy
import sys
sys.path.append('..')
from ocd.models.permutation.utils import all_permutations
# import sys
# import ocd
# import random
device = 'cpu'
# create a random permutation of n numbers

data_config = yaml.safe_load(
    open("../experiments/intervention/data/parametric_laplace_modulated_25_1000_fork_sin_plus_x.yml", "r")
)
model_config = yaml.safe_load(open("../experiments/intervention/model-perm.yml", "r"))

n = data_config["init_args"]["dataset_args"]["scm_generator_args"]["graph_generator_args"]["n"]
model_config["init_args"]["model_args"]["in_features"] = n

# setup fixed perm
enforce_perm = torch.randperm(n).tolist()
data_config["init_args"]["dataset_args"]["scm_generator_args"]["graph_generator_args"][
    "enforce_ordering"
] = enforce_perm
model_config["init_args"]["model_args"]["ordering"] = enforce_perm
model_config["init_args"]["model_args"]["scale_transform_s_args"] = dict(post_act_scale=5.0, normalization=None)
model_config["init_args"]["model_args"]["permutation_learner_args"] = dict(
    # num_hard_samples=100,
    permutation_type="hybrid-sparse-map-simulator",
    # num_samples=20,
    # buffer_replace_prob=0.5,
    # buffer_replay_prob=0.5,
    # buffer_size=200,
)

# trainer_config = yaml.safe_load(open("experiments/intervention/trainer.yml", "r"))
# trainer_config["max_epochs"] = 400
# trainer_config['accelerator'] = device
# trainer_config["enable_progress_bar"] = True
# trainer_config["enable_model_summary"] = True
# grad_clip_val = 1000

# dm = ltb.DataModule(**data_config["init_args"])
# model = dy.eval(model_config["class_path"])(**model_config["init_args"])
# model = model.to(device)
# trainer = lightning.Trainer(
#     **trainer_config,
#     callbacks=[lightning.pytorch.callbacks.LearningRateMonitor()],
#     gradient_clip_val=1.0,
#     gradient_clip_algorithm="value",
# )
# model

In [2]:
from lightning import seed_everything

args = dict(
    # num_hard_samples=100,
    permutation_type="hybrid-sparse-map-simulator",
    # permutation_type='soft'
)
batch_size = 128

model_config["init_args"]["model_args"]["permutation_learner_args"] = args
seed_everything(42)
model_config["init_args"]["model_args"]["permutation_learner_cls"] = "ocd.models.permutation.LearnablePermutation"
model_new = dy.eval(model_config["class_path"])(**model_config["init_args"])
model_config["init_args"]["model_args"][
    "permutation_learner_cls"
] = "ocd.models.permutation.LegacyLearnablePermutation"
model_old = dy.eval(model_config["class_path"])(**model_config["init_args"])
model_new.model.permutation_model.gamma = model_old.model.permutation_model.gamma
model_new.model.permutation_model, model_old.model.permutation_model

Global seed set to 42


(LearnablePermutation(num_features=25, permutation_type=hybrid-sparse-map-simulator),
 LegacyLearnablePermutation(num_features=25))

In [3]:
seed_everything(42)
new_results1 = model_new.model.permutation_model(batch_size)
seed_everything(42)
new_results2 = model_new.model.permutation_model(batch_size)
print((new_results1["soft_perm_mat"] - new_results2["soft_perm_mat"]).abs().max())
print((new_results1["hard_perm_mat"] - new_results2["hard_perm_mat"]).abs().max().max())

Global seed set to 42
Global seed set to 42


tensor(0., grad_fn=<MaxBackward1>)
tensor(0.)


In [4]:
seed_everything(42)
old_results1 = model_old.model.permutation_model(batch_size)
seed_everything(42)
old_results2 = model_old.model.permutation_model(batch_size)
print((old_results1["soft_perm_mat"] - old_results2["soft_perm_mat"]).abs().max())
print((old_results1["hard_perm_mat"] - old_results2["hard_perm_mat"]).abs().max().max())

Global seed set to 42
Global seed set to 42


tensor(0., grad_fn=<MaxBackward1>)
tensor(0.)


In [5]:
print(
    (old_results1["soft_perm_mat"] - new_results1["soft_perm_mat"]).abs().max(),
    new_results1["soft_perm_mat"].shape,
    old_results1["soft_perm_mat"].shape,
)
print(
    (old_results1["hard_perm_mat"] - new_results1["hard_perm_mat"]).abs().max().max(),
    new_results1["hard_perm_mat"].shape,
    old_results1["hard_perm_mat"].shape,
)

tensor(0., grad_fn=<MaxBackward1>) torch.Size([128, 25, 25]) torch.Size([128, 25, 25])
tensor(0.) torch.Size([128, 25, 25]) torch.Size([128, 25, 25])


In [2]:
from lightning import seed_everything

args = dict(
    # num_hard_samples=100,
    permutation_type="hard",
    # permutation_type='soft'
)
batch_size = 128

model_config["init_args"]["model_args"]["permutation_learner_args"] = args
seed_everything(42)
model_config["init_args"]["model_args"]["permutation_learner_cls"] = "ocd.models.permutation.LearnablePermutation"
model_new = dy.eval(model_config["class_path"])(**model_config["init_args"])
model_config["init_args"]["model_args"][
    "permutation_learner_cls"
] = "ocd.models.permutation.LegacyLearnablePermutation"
model_old = dy.eval(model_config["class_path"])(**model_config["init_args"])
model_new.model.permutation_model.gamma = model_old.model.permutation_model.gamma

Global seed set to 42


In [3]:
seed_everything(42)
new_results1 = model_new.model.permutation_model(batch_size)
seed_everything(42)
new_results2 = model_new.model.permutation_model(batch_size)
print((new_results1["perm_mat"] - new_results2["perm_mat"]).abs().max())

Global seed set to 42
Global seed set to 42


tensor(0.)


In [4]:
seed_everything(42)
old_results1 = model_old.model.permutation_model(batch_size)
seed_everything(42)
old_results2 = model_old.model.permutation_model(batch_size)
print((old_results1["perm_mat"] - old_results2["perm_mat"]).abs().max(), "a")

Global seed set to 42
Global seed set to 42


tensor(0.) a


In [5]:
print(
    (old_results1["perm_mat"] - new_results1["perm_mat"]).abs().max(),
    new_results1["perm_mat"].shape,
    old_results1["perm_mat"].shape,
)

tensor(0.) torch.Size([128, 25, 25]) torch.Size([128, 25, 25])


In [2]:
from lightning import seed_everything

args = dict(
    # num_hard_samples=100,
    permutation_type="soft",
    # permutation_type='soft'
)
batch_size = 128

model_config["init_args"]["model_args"]["permutation_learner_args"] = args
seed_everything(42)
model_config["init_args"]["model_args"]["permutation_learner_cls"] = "ocd.models.permutation.LearnablePermutation"
model_new = dy.eval(model_config["class_path"])(**model_config["init_args"])
model_config["init_args"]["model_args"][
    "permutation_learner_cls"
] = "ocd.models.permutation.LegacyLearnablePermutation"
model_old = dy.eval(model_config["class_path"])(**model_config["init_args"])
model_new.model.permutation_model.gamma = model_old.model.permutation_model.gamma

Global seed set to 42


In [3]:
seed_everything(42)
new_results1 = model_new.model.permutation_model(batch_size)
seed_everything(42)
new_results2 = model_new.model.permutation_model(batch_size)
print((new_results1["perm_mat"] - new_results2["perm_mat"]).abs().max())

Global seed set to 42
Global seed set to 42


tensor(0., grad_fn=<MaxBackward1>)


In [4]:
seed_everything(42)
old_results1 = model_old.model.permutation_model(batch_size)
seed_everything(42)
old_results2 = model_old.model.permutation_model(batch_size)
print((old_results1["perm_mat"] - old_results2["perm_mat"]).abs().max())

Global seed set to 42
Global seed set to 42


tensor(0., grad_fn=<MaxBackward1>)


In [5]:
print(
    (old_results1["perm_mat"] - new_results1["perm_mat"]).abs().max(),
    new_results1["perm_mat"].shape,
    old_results1["perm_mat"].shape,
)

tensor(0., grad_fn=<MaxBackward1>) torch.Size([128, 25, 25]) torch.Size([128, 25, 25])


In [2]:
from lightning import seed_everything

args = dict(
    # num_hard_samples=100,
    permutation_type="hybrid-quantization",
    hard_from_softs=True,
)
batch_size = 128

model_config["init_args"]["model_args"]["permutation_learner_args"] = args
seed_everything(42)
model_config["init_args"]["model_args"]["permutation_learner_cls"] = "ocd.models.permutation.LearnablePermutation"
model_new = dy.eval(model_config["class_path"])(**model_config["init_args"])
model_config["init_args"]["model_args"]["permutation_learner_args"].pop("hard_from_softs")
model_config["init_args"]["model_args"][
    "permutation_learner_cls"
] = "ocd.models.permutation.LegacyLearnablePermutation"
model_old = dy.eval(model_config["class_path"])(**model_config["init_args"])
model_new.model.permutation_model.gamma = model_old.model.permutation_model.gamma

Global seed set to 42


In [3]:
seed_everything(42)
new_results1 = model_new.model.permutation_model(batch_size)
seed_everything(42)
new_results2 = model_new.model.permutation_model(batch_size)
print((new_results1["perm_mat"] - new_results2["perm_mat"]).abs().max())

Global seed set to 42
Global seed set to 42


tensor(0., grad_fn=<MaxBackward1>)


In [4]:
seed_everything(42)
old_results1 = model_old.model.permutation_model(batch_size)
seed_everything(42)
old_results2 = model_old.model.permutation_model(batch_size)
print((old_results1["perm_mat"] - old_results2["perm_mat"]).abs().max())

Global seed set to 42
Global seed set to 42


tensor(0., grad_fn=<MaxBackward1>)


In [5]:
print(
    (old_results1["perm_mat"] - new_results1["perm_mat"]).abs().max(),
    new_results1["perm_mat"].shape,
    old_results1["perm_mat"].shape,
)

tensor(0., grad_fn=<MaxBackward1>) torch.Size([128, 25, 25]) torch.Size([128, 25, 25])
